In [1]:
from pathlib import Path
import tifffile
import os
import numpy as np
from tqdm import tqdm
from stardist.models import StarDist3D
from utils_stardist import get_gpu_details, list_images, read_image, extract_nuclei_stack, segment_nuclei_3d

get_gpu_details()

Device name: /device:GPU:0
Device type: GPU
GPU model: device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [2]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/test_data")

# Construct ROI path from directory_path above
roi_path = directory_path / "ROIs"

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['..\\raw_data\\test_data\\HI1_CONTRA_M8_S6_TR1.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S6_TR2.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S7_TR1.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S7_TR2.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S6_TR1.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S6_TR2.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S7_TR1.czi',
 '..\\raw_data\\test_data\\HI2_CONTRA_M10_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI2_CONTRA_M10_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI2_IPSI_M10_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI2_IPSI_M10_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI3_CONTRA_M11_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI3_CONTRA_M11_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI3_IPSI_M11_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI3_IPSI_M11_S10_TR2.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M6_S11_TR1.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M6_S11_TR2.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M7_S11_TR1.czi',
 '..\\raw_data\\t

<h3>Define your nuclei channel and your nuclear segmentation parameters</h3>

Modify the values for <code>slicing factor</code>, <code>nuclei_channel</code>, <code>segmentation_type</code> and your Stardist <code>model_name</code>


In [3]:
# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3

n_tiles=(6,6,3)

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "3D"

# Nuclear segmentation model type ("Stardist")
# Choose your Stardist fine-tuned model (model_name) from stardist_models folder
model_name = "MEC0.1"

# Model loading 
model = StarDist3D(None, name=model_name, basedir='stardist_models')

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.583933, nms_thresh=0.3.


<h3>Predict nuclei labels and store them as .tiff files</h3>

In [4]:
# List of subfolder names
try:
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]
    print(f"The following regions of interest will be analyzed: {roi_names}")
except FileNotFoundError:
    roi_names = ["full_image"]

for image in tqdm(images):
    for roi_name in roi_names:
        # Check if the prediction has already been generated
        try:
            file_path = Path(image)
            filename = file_path.stem
            tifffile.imread(directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name / f"{filename}.tiff")
            print(f"\nWARNING: Nuclei predictions already found for: {filename} ROI: {roi_name}")
            print("Make sure nuclei labels were generated using the same settings.")
            continue  # Skip to the next roi_name if the prediction exists
        except FileNotFoundError:
            pass  # Proceed to generate predictions if the file is not found

        # Read image stack and extract filename
        img, filename = read_image(image, slicing_factor)

        # Slice the nuclei stack
        nuclei_img = extract_nuclei_stack(img, nuclei_channel)

        print(f"Generating nuclei predictions for {roi_name} ROI")

        # Construct path to read ROI
        roi_path = directory_path / "ROIs" / roi_name / f"{filename}.tiff"

        try:
            # Read the .tiff files containing the user-defined ROIs
            roi = tifffile.imread(roi_path)

            # We will create a mask where roi is greater than or equal to 1
            mask = (roi >= 1).astype(np.uint8)

            # 3D segmentation logic, extend 2D mask across the entire stack volume
            if segmentation_type == "3D":
                # Extract the number of z-slices to extend the mask
                slice_nr = img.shape[1]

                # Extend the mask across the entire volume
                mask = np.tile(mask, (slice_nr, 1, 1))

                # Apply the mask to nuclei_img and marker_img, setting all other pixels to 0
                masked_nuclei_img = np.where(mask, nuclei_img, 0)
            else:
                # Apply the mask to nuclei_img and marker_img, setting all other pixels to 0
                masked_nuclei_img = np.where(mask, nuclei_img, 0)

        except FileNotFoundError:
            # If no ROI is saved the script will predict nuclei in the entire nuclei_img input
            masked_nuclei_img = nuclei_img

        # Segment nuclei and return labels
        nuclei_labels = segment_nuclei_3d(masked_nuclei_img, model, n_tiles)

        # Save nuclei labels as .tiff files to reuse them later
        # Create nuclei_predictions directory if it does not exist
        try:
            os.makedirs(directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name)
        except FileExistsError:
            pass

        # Construct path to store
        nuclei_preds_path = directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name / f"{filename}.tiff"

        # Save nuclei labels as .tiff
        tifffile.imwrite(nuclei_preds_path, nuclei_labels)

print("\nNuclei prediction completed")



The following regions of interest will be analyzed: ['CA', 'DG']


  0%|          | 0/23 [00:00<?, ?it/s]



Image analyzed: HI1_CONTRA_M8_S6_TR1
Original Array shape: (4, 14, 3803, 2891)
Compressed Array shape: (4, 14, 3803, 2891)
Generating nuclei predictions for CA ROI


100%|██████████| 18/18 [00:13<00:00,  1.34it/s]




Image analyzed: HI1_CONTRA_M8_S6_TR1
Original Array shape: (4, 14, 3803, 2891)
Compressed Array shape: (4, 14, 3803, 2891)
Generating nuclei predictions for DG ROI


...



Image analyzed: SHAM1_IPSI_M7_S11_TR2
Original Array shape: (4, 15, 2883, 2889)
Compressed Array shape: (4, 15, 2883, 2889)
Generating nuclei predictions for CA ROI


100%|██████████| 18/18 [00:05<00:00,  3.27it/s]




Image analyzed: SHAM1_IPSI_M7_S11_TR2
Original Array shape: (4, 15, 2883, 2889)
Compressed Array shape: (4, 15, 2883, 2889)
Generating nuclei predictions for DG ROI


100%|██████████| 23/23 [46:21<00:00, 120.93s/it]


Nuclei prediction completed
